In [1]:
import json
import time

import flax
import flax.serialization
from luxai_s3.params import EnvParams
from luxai_s3.state import EnvState, serialize_env_actions, serialize_env_states
import jax
import jax.numpy as jnp

from luxai_s3.env import LuxAIS3Env

# from luxai_s3.wrappers import RecordEpisode

# Create the environment
env = LuxAIS3Env(auto_reset=False)
env_params = EnvParams(map_type=0, max_steps_in_match=50)

# Initialize a random key
key = jax.random.key(0)

# Reset the environment
key, reset_key = jax.random.split(key)
obs, state = env.reset(reset_key, params=env_params)
# Take a random action
key, subkey = jax.random.split(key)

env_params

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


EnvParams(max_steps_in_match=50, map_type=0, map_width=24, map_height=24, num_teams=2, match_count_per_episode=5, max_units=16, init_unit_energy=100, min_unit_energy=0, max_unit_energy=400, unit_move_cost=2, spawn_rate=5, unit_sap_cost=10, unit_sap_range=4, unit_sap_dropoff_factor=0.5, unit_energy_void_factor=0.125, max_energy_nodes=6, max_energy_per_tile=20, min_energy_per_tile=-20, max_relic_nodes=6, relic_config_size=5, fog_of_war=True, unit_sensor_range=2, nebula_tile_vision_reduction=1, nebula_tile_energy_reduction=0, nebula_tile_drift_speed=-0.05, energy_node_drift_speed=0.02, energy_node_drift_magnitude=5)

In [2]:
for _ in range(5):
    key, subkey = jax.random.split(key)
    action = env.action_space(env_params).sample(subkey)
    obs, state, reward, terminated, truncated, info = env.step(
        subkey, state, action, params=env_params
    )

In [7]:
from typing import Any, Dict


class Agent:
    def __init__(self, player: str, env_cfg: Dict[str, Any]) -> None:
        self.player = player
        self.opp_player = "player_1" if self.player == "player_0" else "player_0"
        self.team_id = 0 if self.player == "player_0" else 1
        self.opp_team_id = 1 if self.team_id == 0 else 0
        np.random.seed(0)
        self.env_cfg = env_cfg

        self.map = Graph(env_cfg["map_width"], env_cfg["map_height"])

    def act(self, step: int, obs: Dict[str, Any], remainingOverageTime: int = 60):
        """implement this function to decide what actions to send to each available unit.

        step is the current timestep number of the game starting from 0 going up to max_steps_in_match * match_count_per_episode - 1.
        """

        unit_mask = np.array(obs["units_mask"][self.team_id])
        unit_positions: NDArray[np.int16] = np.array(
            obs["units"]["position"][self.team_id]
        )[unit_mask]
        return 0

In [8]:
def direction_to(src, target):
    ds = target - src
    dx = ds[0]
    dy = ds[1]
    if dx == 0 and dy == 0:
        return 0
    if abs(dx) > abs(dy):
        if dx > 0:
            return 2
        else:
            return 4
    else:
        if dy > 0:
            return 3
        else:
            return 1


def manhattan_distance(start_node: ArrayLike, end_node: ArrayLike) -> int:
    return abs(start_node[0] - end_node[0]) + abs(start_node[0] - end_node[0])

NameError: name 'ArrayLike' is not defined

In [9]:
from argparse import Namespace
from dataclasses import dataclass
from typing import Any, Dict
import numpy as np
import networkx as nx
import numpy as np
from numpy.typing import ArrayLike, NDArray


@dataclass(frozen=True)
class Coordinate:
    x: int
    y: int

    def __getitem__(self, index: int):
        if index == 0:
            return self.x
        elif index == 1:
            return self.y
        else:
            raise IndexError("Index out of range. Use 0 for 'x' and 1 for 'y'.")


class Graph:
    def __init__(self, map_width: int, map_height: int):
        self.map: nx.DiGraph = nx.grid_2d_graph(
            map_width, map_height, periodic=False, create_using=nx.DiGraph()
        )

    def update_cost(self, node: ArrayLike, cost: float):
        nx.set_edge_attributes(self.map, cost, "cost")

    def map_shortest_path_to_action(
        self, start_node: ArrayLike, end_node: ArrayLike
    ) -> int:
        next_node = nx.astar_path(
            self.map, start_node, end_node, heuristic=manhattan_distance
        )[1]

        return direction_to(np.array(start_node), np.array(next_node))

In [10]:
G = Graph(24, 24)
{e: 0 for e in G.map.edges()}
G.map.add_node((0, 0))

In [14]:
len(G.map.in_edges())

2208

In [30]:
obs_player_0 = obs["player_0"]

In [ ]:
visible_tiles_indices = jax.numpy.indices(obs_player_0.sensor_mask.shape)[
    :, obs_player_0.sensor_mask
]
visible_energies = obs_player_0.map_features.energy[obs_player_0.sensor_mask]
for i in range(visible_tiles_indices.shape[1]):
    node_location = tuple(visible_tiles_indices[:, i])
    print(node_location, visible_energies)
    cost = visible_energies[node_location[0], node_location[1]]
    G.map.add_node(node_location, cost=cost)

(Array(0, dtype=int32), Array(0, dtype=int32)) [ 5  4  2  0  4  2  0 -1  2  0 -1  0]


IndexError: Too many indices: 1-dimensional array indexed with 2 regular indices.

In [23]:
from luxai_s3.state import EnvObs


def env_params_to_dict(env_params: EnvParams) -> Dict[str, Any]:
    return {
        "map_width": env_params.map_width,
        "map_height": env_params.map_height,
        "max_steps_in_match": env_params.max_steps_in_match,
    }


def obs_to_dict(obs: EnvObs) -> Dict[str, Any]:
    return {
        "units": {
            "position": obs.units.position,
            "energy": obs.units.energy,
        },
        "units_mask": obs.units_mask,
        "sensor_mask": obs.sensor_mask,
        "map_features": {
            "energy": obs.map_features.energy,
            "tile_type": obs.map_features.tile_type,
        },
        "relic_nodes": obs.relic_nodes,
        "relic_nodes_mask": obs.relic_nodes_mask,
        "team_points": obs.team_points,
        "team_wins": obs.team_wins,
        "steps": obs.steps,
        "match_steps": obs.match_steps,
    }


Agent("player_0", env_params_to_dict(env_params)).act(0, obs_to_dict(obs["player_0"]))

2208


0